# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("wikipedia_train3.csv")
test = pd.read_csv("wikipedia_test3.csv")

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [3]:
train = train.fillna(0)
test = test.fillna(0)

In [4]:
train["date"] = train["date"].astype('<M8[ns]')
test["date"] = test["date"].astype('<M8[ns]')

In [5]:
def add_f(df):
    df['Month']     = df["date"].dt.month
    df['Day']       = df["date"].dt.day
    df['DayOfWeek'] = df["date"].dt.dayofweek
    return df

In [6]:
add_f(train);
add_f(test);

In [47]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [48]:
train['lang'] = train.Page.map(get_language)
test['lang'] = test.Page.map(get_language)

In [50]:
test['lang'].value_counts()

en    487010
es     80414
ja     70804
ru     36642
de     24986
fr     15562
zh     12214
na      4898
Name: lang, dtype: int64

In [52]:
train = pd.concat([train, pd.get_dummies(train['lang'], prefix="lang")],axis=1)
test = pd.concat([test, pd.get_dummies(test['lang'], prefix="lang")],axis=1)
                

In [54]:
train.drop(['lang'],axis=1, inplace=True)
test.drop(['lang'],axis=1, inplace=True)

In [55]:
train.head()

,Page,date,Visits,Month,Day,DayOfWeek,lang_de,lang_en,lang_es,lang_fr,lang_ja,lang_na,lang_ru,lang_zh
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0,1,1,4,0,0,0,0,0,0,0,1
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0,1,1,4,0,0,0,0,0,0,0,1
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0,1,1,4,0,0,0,0,0,0,0,1
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0,1,1,4,0,0,0,0,0,0,0,1
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0,1,1,4,0,0,0,1,0,0,0,0


In [8]:
test["date"].unique().shape[0]/(train["date"].unique().shape[0]+ test["date"].unique().shape[0])

0.20261437908496732

In [9]:
test.head()

,Page,date,Visits,Month,Day,DayOfWeek
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-09-10,640.0,9,10,5
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-09-10,5609.0,9,10,5
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-09-10,3121.0,9,10,5
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-09-10,362.0,9,10,5
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-09-10,533.0,9,10,5


In [10]:
import math

In [72]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [11]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [12]:
from dateutil.relativedelta import relativedelta

In [13]:
lastday = max(train['date'])- relativedelta(days=21)
lastday

Timestamp('2016-08-10 00:00:00')

In [14]:
train_train, train_holdout = train[train['date']<(lastday - relativedelta(days=30))], \
                                train[train['date']>=(lastday - relativedelta(days=30))]

In [15]:
train_train.shape[0]/(train_train.shape[0] +train_holdout.shape[0])

0.7868852459016393

In [16]:
def model_15days(df_tr,df_h):
    median_baseline = df_tr.loc[train["date"] > (max(df_tr['date']) - relativedelta(days=15))].groupby(['Page']).median().reset_index()
    median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)
    return df_h.merge(median_baseline, on="Page", how='left')


In [17]:
pandas_smape(model_15days(train_train,train_holdout))

39.31895168678857

In [18]:
pandas_smape(model_15days(train_holdout,test))

40.007012702044406

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [19]:
def create_validation(df, start_date):
    return df[(df['date'] >= pd.to_datetime(start_date) - relativedelta(months=3)) & \
              (df['date'] <  pd.to_datetime(start_date) - relativedelta(months=0))].index, \
           df[(df['date'] >= pd.to_datetime(start_date) - relativedelta(months=0)) & \
              (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index

In [20]:
train_dates = ['2016-04-10', '2016-05-10', '2016-06-10']

In [85]:
### Your validation iterator
myCViterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(train, i)
    myCViterator.append( (trainIndices, valIndices) )

In [24]:
scores = []
for train_indexes, val_indexes in myCViterator:
    val_df_pred = model_15days(train.iloc[train_indexes].copy(), 
                                        train.iloc[val_indexes].copy())
    score = pandas_smape(val_df_pred)
    scores.append(score)
print( scores, np.mean(scores))

[39.968622328046635, 40.848838633224524, 42.603892486759456] 41.14045114934354


In [67]:
X = train.drop(['Visits','date', "Page"], axis = 1)
y = train['Visits']

X_test = test.drop(['Page','Visits','date'], axis = 1)
y_test = test['Visits']

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [58]:
from sklearn import linear_model

In [ ]:
#from sklearn import linear_model

In [59]:
clf = linear_model.SGDRegressor( average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.25, learning_rate='invscaling',
       loss='squared_loss', 
       penalty='elasticnet', power_t=0.25, random_state=42, shuffle=True,
        verbose=0, warm_start=False)

In [60]:
clf.get_params().keys()

dict_keys(['alpha', 'average', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'verbose', 'warm_start'])

In [61]:
params = {'alpha': [ 0.001,0.1, 1, 10]}
#params = {'penalty' : ['l1','l2']}

In [64]:
grid_search = GridSearchCV(clf, param_grid=params, cv=myCViterator)

In [68]:
%time grid_search.fit(X,y)

/Users/max/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/max/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/max/anaconda3/l

CPU times: user 23.1 s, sys: 2.93 s, total: 26 s
Wall time: 17.7 s


GridSearchCV(cv=[(Int64Index([ 106335,  106336,  106337,  106338,  106339,  106340,  106341,
             106342,  106343,  106344,
            ...
            1181490, 1181491, 1181492, 1181493, 1181494, 1181495, 1181496,
            1181497, 1181498, 1181499],
           dtype='int64', length=1075165), Int64In...2622925, 2622926,
            2622927, 2622928, 2622929],
           dtype='int64', length=720715))],
       error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.25, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None,
       penalty='elasticnet', power_t=0.25, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.001, 0.1, 1, 10]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [69]:
grid_search.best_params_

{'alpha': 10}

In [70]:
y_pred = pd.DataFrame(grid_search.predict(X_test))

In [77]:
a = pd.concat([y_pred, y],axis=1)

In [78]:
a.columns = ["pred_Visits","Visits"]

In [80]:
pandas_smape(a)

169.92019219005

In [99]:
scores = []
for train_indexes, val_indexes in myCViterator:
    val_df_pred = pd.DataFrame(grid_search.predict( X.iloc[val_indexes].copy()))
    arr = pd.concat([val_df_pred, y],axis=1)
    arr.columns = ["pred_Visits","Visits"]
    score = pandas_smape(arr)
    scores.append(score)
print( scores, np.mean(scores))

[169.11785256689964, 169.36533877786482, 169.62447650373173] 169.3692226161654


In [111]:
# clf = linear_model.SGDRegressor( average=False, epsilon=0.1, eta0=0.01,fit_intercept=True,\
#                                 l1_ratio=0.25, learning_rate='invscaling', loss='squared_loss',\
#                                 penalty='elasticnet', power_t=0.25, random_state=42, shuffle=True,\
#                                 verbose=0, warm_start=False)  
# scores= [] 
# features = ['Month','Day','DayOfWeek', 'date'] 
# for page in train.Page.unique()[:10]:
#     train_features = train.loc[train.Page == page, features]
#     train_target = train.loc[train.Page == page,'Visits']

#     myCViterator = []     
#     for i in train_dates:
#         trainIndices, valIndices = create_validation(train_features, i)         
#         myCViterator.append( (trainIndices, valIndices) )             
#         print(trainIndices, valIndices)          
#         grid_search = GridSearchCV(clf, param_grid=params, cv=myCViterator)       
#         grid_search.fit(train_features, train_target)     
#         cv_score = np.mean(cross_val_score(grid_search, train_features, train_target, scoring='roc_auc'))          
#         print('CV score for class {} is {}'.format(page, cv_score))     
#         scores.append(cv_score)


Int64Index([ 106335,  118150,  129965,  141780,  153595,  165410,  177225,
             189040,  200855,  212670,  224485,  236300,  248115,  259930,
             271745,  283560,  295375,  307190,  319005,  330820,  342635,
             354450,  366265,  378080,  389895,  401710,  413525,  425340,
             437155,  448970,  460785,  472600,  484415,  496230,  508045,
             519860,  531675,  543490,  555305,  567120,  578935,  590750,
             602565,  614380,  626195,  638010,  649825,  661640,  673455,
             685270,  697085,  708900,  720715,  732530,  744345,  756160,
             767975,  779790,  791605,  803420,  815235,  827050,  838865,
             850680,  862495,  874310,  886125,  897940,  909755,  921570,
             933385,  945200,  957015,  968830,  980645,  992460, 1004275,
            1016090, 1027905, 1039720, 1051535, 1063350, 1075165, 1086980,
            1098795, 1110610, 1122425, 1134240, 1146055, 1157870, 1169685],
           dtype='int64'

IndexError: positional indexers are out-of-bounds

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [25]:
train_f = pd.read_csv('train_french.csv')
test_f = pd.read_csv('test_french.csv')

In [26]:
train_f.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [27]:
test_f.shape[0]/(train_f.shape[0]+ test_f.shape[0])

0.275278293135436

In [28]:
train_f= train_f.fillna(0)

In [29]:
test_f= test_f.fillna(0)

In [30]:
train_f.describe()

,code_postal,nb_pieces,biens_nomades,societe,domaine_activite,age_societaire,objets_precieux,pres_contrat_auto,formule,sit_juridique,...,var2,var3,var4,var5,var6,var7,var8,var10,var11,prime_tot_ttc
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,53290.034095,2.913660,585.490000,1.378265,19.73155,46.110705,1.04169,0.744895,1.566090,0.645205,...,0.737760,0.123480,100.609715,82.822945,0.387730,0.056710,1.524510,16.408595,1.777370,96.497965
std,27916.895626,1.313912,328.811497,0.484955,25.14893,16.677828,0.19988,0.435921,0.495614,0.707056,...,1.059861,0.328988,58.567455,27.343636,0.487234,0.231288,1.335844,11.963039,0.416013,32.965500
min,1001.000000,1.000000,0.000000,1.000000,0.00000,18.000000,1.00000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,1.000000,0.000000,1.000000,32.060000
25%,31526.000000,2.000000,500.000000,1.000000,2.00000,32.000000,1.00000,0.000000,1.000000,0.000000,...,0.000000,0.000000,50.000000,48.000000,0.000000,0.000000,1.000000,10.000000,2.000000,73.480000
50%,57463.000000,3.000000,500.000000,1.000000,9.00000,43.000000,1.00000,1.000000,2.000000,1.000000,...,0.000000,0.000000,101.000000,100.000000,0.000000,0.000000,1.000000,12.000000,2.000000,89.930000
75%,75119.000000,4.000000,1000.000000,2.000000,34.00000,58.000000,1.00000,1.000000,2.000000,1.000000,...,1.000000,0.000000,151.000000,100.000000,1.000000,0.000000,1.000000,30.000000,2.000000,112.700000
max,97801.000000,5.000000,2000.000000,2.000000,99.00000,100.000000,2.00000,1.000000,2.000000,3.000000,...,9.000000,1.000000,300.000000,100.000000,1.000000,1.000000,6.000000,80.000000,2.000000,469.670000


In [33]:
!pip install pandas_profiling

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [100]:
import pandas_profiling as pp

In [101]:
pp.ProfileReport(train_f)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,18
Categorical,0
Boolean,11
Date,0
Text (Unique),0
Rejected,0


In [102]:
pp.ProfileReport(test_f)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,18
Categorical,0
Boolean,11
Date,0
Text (Unique),0
Rejected,0


In [35]:
from sklearn.model_selection import train_test_split

In [36]:
train_f.columns

Index(['code_postal', 'nb_pieces', 'veranda', 'biens_nomades', 'sit_fam',
       'profession', 'nature_lri', 'societe', 'domaine_activite',
       'niv_urbanisation', 'age_societaire', 'objets_precieux',
       'pres_contrat_auto', 'formule', 'sit_juridique', 'surf_habitat',
       'patrimoine_mob', 'var1', 'var2', 'var3', 'var4', 'var5', 'var6',
       'var7', 'var8', 'var9', 'var10', 'var11', 'prime_tot_ttc'],
      dtype='object')

In [37]:
from sklearn.ensemble import RandomForestRegressor

In [38]:

binary_cols = sorted([
    'veranda','sit_fam' ,'formule','objets_precieux','pres_contrat_auto', 'var3', 'var7', 'var9', 'var11',
])
categorical_cols = sorted([
    'nature_lri', 'niv_urbanisation', 'profession'
])

In [39]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [40]:
from collections import defaultdict

# function to encode categorical data


def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))


# label encode data (categorical + binary)
__encode_categorical(df_list=[train_f], cat_cols=categorical_cols+binary_cols)
__encode_categorical(df_list=[test_f], cat_cols=categorical_cols+binary_cols)

# define useful feature columns to be used for training
# (union of all columns discussed above)
columns_to_use = binary_cols + categorical_cols

In [41]:
x_train, x_hold, y_train, y_hold = train_test_split(train_f.drop('prime_tot_ttc', axis=1),train_f['prime_tot_ttc'],
                                                   test_size=0.3, 
                                                   random_state=42)

In [42]:
regr = RandomForestRegressor(max_depth=6, random_state=42)

In [43]:
[round(np.sqrt(x_train.columns.shape[0])),round(x_train.columns.shape[0])*0.5, round(x_train.columns.shape[0]*0.75),x_train.columns.shape[0] ]

[5.0, 14.0, 21, 28]

In [44]:
params = {
    #'n_estimators' : [3,5,8,10, 13],
    'max_depth': [3, 5,  9, 11, 25],
    #'min_samples_split': [2, 5, 9, 15, 25],
    'max_features': [5,14,21,28],
    'min_samples_leaf': [1, 3, 5, 7, 11]}

In [45]:
from sklearn import ensemble

In [46]:
params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 5,'max_features' : 21,
          'learning_rate': 0.01, 'loss': 'ls'}

In [47]:
clf = ensemble.GradientBoostingRegressor(**params,random_state=42)

In [48]:
clf.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=6, max_features=21,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=5, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False)

In [54]:
def mape_score(y, y_pred):
    return np.sum(100. / y.shape[0] * np.abs((y - y_pred) / y))

In [55]:
y_val_pred_b = clf.predict(x_hold)
y_test_pred_b = clf.predict(test_f.drop('prime_tot_ttc', axis=1))

In [56]:
mape_score(y_hold, y_val_pred_b)

13.446208395170665

In [57]:
mape_score(test_f['prime_tot_ttc'], y_test_pred_b)

13.499810714438311